In [1]:
import cftime
import glob
import pandas as pd
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
from scipy import stats

import mission_tools

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
mission_list = glob.glob('../microSWIFT_data/final_dataset/mission_*.nc')

hs_all = []
tm_all = []
mean_dir_all = []
xs_gamma_loc_all = []
water_level_all = []
mission_number = []
mission_break_depth_all = []
mission_time_all = []
mission_spectra_all = []
mission_dir_spectra_all = []
freq_all = []
dir_bins_all = []
wind_spd_all = []
wind_dir_all = []
water_level_all = []

gamma = 0.35
awac_file = '../microSWIFT_data/FRFdata/FRF-ocean_waves_8m-array_202110.nc'
wind_file = '../microSWIFT_data/FRFdata/FRF-met_wind_derived_202110.nc'
bathy_file = '../microSWIFT_data/FRFdata/FRF_geomorphology_DEMs_surveyDEM_20211021.nc'
bathy_dataset = nc.Dataset(bathy_file)
xFRF_grid, yFRF_grid = np.meshgrid(bathy_dataset['xFRF'][:],bathy_dataset['yFRF'][:])
bathy = bathy_dataset['elevation'][0,:,:]
waterlevel_file = '../microSWIFT_data/FRFdata/FRF-ocean_waterlevel_eopNoaaTide_202110.nc'

for mission_nc in mission_list:
    mission_dataset = nc.Dataset(mission_nc, mode='r')

    mission_time = np.median(mission_dataset['time'][:])

    # Compute depth that waves should be breaking in 
    mission_hs = mission_tools.closest_awac_sig_wave_height(mission_time, awac_file)
    mission_tm = mission_tools.closest_awac_Tm(mission_time, awac_file)
    mission_mean_dir = mission_tools.closest_awac_dir(mission_time, awac_file)
    mission_spectra, freq = mission_tools.closest_awac_spectra(mission_time, awac_file)
    mission_dir_spectra, _, dirs = mission_tools.closest_awac_dir_spectra(mission_time, awac_file)
    wind_spd, wind_dir = mission_tools.closest_wind_spd_and_dir(mission_time, wind_file)
    water_level = mission_tools.closest_waterlevel(mission_time, waterlevel_file )

    # Compute 
    mission_break_depth = mission_hs / gamma
    mission_xs_gamma_location = np.interp(-mission_break_depth, np.flip(np.nanmean(bathy, axis=0)), np.flip(bathy_dataset['xFRF'][:]))

    # Save all values
    hs_all.append(mission_hs)
    tm_all.append(mission_tm)
    mean_dir_all.append(mission_mean_dir)
    xs_gamma_loc_all.append(mission_xs_gamma_location)
    mission_break_depth_all.append(mission_break_depth)
    mission_time_all.append(mission_time)
    mission_spectra_all.append(mission_spectra.tolist())
    freq_all.append(freq.tolist())
    dir_bins_all.append(dirs.tolist())
    mission_dir_spectra_all.append(mission_dir_spectra.tolist())
    wind_spd_all.append(wind_spd)
    wind_dir_all.append(wind_dir)
    water_level_all.append(water_level)
    
    # Save mission number
    mission_number.append(int(mission_nc[41:-3]))

/opt/homebrew/Caskroom/mambaforge/base/envs/dunex-venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:784: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)


In [3]:
mission_df = pd.DataFrame(cftime.num2pydate(mission_time_all, units=mission_dataset['time'].units,
                          calendar=mission_dataset['time'].calendar), columns=['time'])
mission_df['mission number'] = mission_number
mission_df['Hs [m]'] = hs_all
mission_df['Tm [s]'] = tm_all
mission_df['Mean Dir [deg]'] = mean_dir_all
mission_df['cross shore gamma location [m]'] = xs_gamma_loc_all
mission_df['break depth'] = mission_break_depth_all
mission_df['freq [hz]'] = freq_all
mission_df['energy density [m^2\hz]'] = mission_spectra_all
mission_df['direction bins [deg]'] = dir_bins_all
mission_df['directional energy density [m^2/hz/deg]'] = mission_dir_spectra_all
mission_df['wind speed [m/s]'] = wind_spd_all
mission_df['wind direction [deg]'] = wind_dir_all
mission_df['water level [m]'] = water_level_all

mission_df

,time,mission number,Hs [m],Tm [s],Mean Dir [deg],cross shore gamma location [m],break depth,freq [hz],energy density [m^2\hz],direction bins [deg],directional energy density [m^2/hz/deg],wind speed [m/s],wind direction [deg],water level [m]
0,2021-10-29 14:33:54.998780,78,2.133724,6.918120,80.728183,532.012197,6.096354,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.05025098354559162, 0.07217063888585135, 0.3...","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[8.774435264058411e-05, 9.57169831963256e-05,...",9.615158,156.11945,0.499
1,2021-10-13 13:19:59.997600,29,1.092724,6.420097,58.598522,263.599684,3.122068,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.013723535890658674, 0.015484145426793838, 0...","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[3.3258882467634976e-05, 3.438745261519216e-0...",7.155667,354.59845,-0.046
2,2021-10-07 14:34:59.998800,9,1.204854,5.407665,63.007655,283.712595,3.442439,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.011685272329470702, 0.01250600889351457, 0....","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[2.324277193110902e-05, 2.483886419213377e-05...",8.829667,37.633804,0.627
3,2021-10-15 16:19:29.998200,39,0.588102,7.199700,80.680591,199.966839,1.680292,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.006249074262052682, 0.006294627318541242, 0...","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[1.1981144780293107e-05, 1.1754053048207425e-...",2.9336667,14.616138,-0.055
4,2021-10-28 15:41:59.998080,68,1.482833,7.567346,58.694913,378.892729,4.236665,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.04060271438553851, 0.041858505216364424, 0....","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[0.00010348302021156996, 0.000102618650998920...",7.126901,36.38933,0.783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,2021-10-29 13:05:59.999280,75,2.160752,6.956811,80.367305,538.717394,6.173577,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.05383202134492086, 0.08968265662088824, 0.6...","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[7.690951315453276e-05, 8.58226339914836e-05,...",11.947333,135.17111,0.348
63,2021-10-11 13:21:29.997960,24,2.162277,6.206282,56.501510,539.095785,6.177935,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.05071478808400436, 0.07569422978112746, 0.1...","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[0.00011699234164552763, 0.000126075814478099...",13.889231,11.561683,0.825
64,2021-10-05 18:18:29.998680,4,0.522822,8.807460,84.789797,183.005448,1.493776,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.008461591649143705, 0.009608970959136827, 0...","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[1.2740909369313158e-05, 1.4296941117208917e-...",3.5843334,131.2877,-0.479
65,2021-10-14 13:07:29.998200,34,0.828214,7.232409,73.107152,232.633715,2.366327,"[0.03999999910593033, 0.04749999940395355, 0.0...","[0.01278272131939542, 0.010030188729890517, 0....","[0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0,...","[[2.4330131054739468e-05, 2.5680119506432675e-...",3.8991666,334.03748,-0.257


In [4]:
mission_df.to_csv('../microSWIFT_data/mission_df.csv')